In [ ]:
!pip uninstall moment

In [8]:
import keras_bitcoin
import dataset
import collections
import pandas as pd
from itertools import permutations, combinations
from sklearn.model_selection import train_test_split
import moment

In [13]:
df = dataset.get_labeled_dataset(number_of_file=10, binary= False, values = [-5, -0.2, 0.2, 5], from_date = "2018-01-01")
print(collections.Counter(df["label"]))

df.head()
print(df.shape)
print(df.head())

Number of files loaded :  12
Counter({1: 3228, 4: 2646, 5: 564, 2: 550, 3: 188})
(7176, 8)
                                                text  label        date  \
0          So how does that make it not a currency?       3  2018-01-03   
1  I didn't say it wasn't a currency, I'm just sa...      3  2018-01-03   
2  I don't think spam has ever been a problem in ...      3  2018-01-03   
3  It was electrum wallet bro ._.\r\nMy transacti...      3  2018-01-03   
4  My laptop has 4 GB Ram, I think it can handle ...      3  2018-01-03   

   score  nb_replies  stickied  label_m1  label_m2  
0      2           1     False         4         5  
1      1           0     False         4         5  
2      1           1     False         4         5  
3      1           0     False         4         5  
4      1           0     False         4         5  


In [14]:
texts = df["text"]
labels = df["label"]
texts_train, texts_test , labels_train, labels_test, vocab_length, max_sentence_size\
= keras_bitcoin.get_train_test_data(texts, labels)

In [285]:
epochs=5
batch_size=None
perfs = {}
dropouts = []

temp = []

function_list =  list(combinations(keras_bitcoin.available_activation_functions, 3))

size = len(function_list)
count = 0
for function in function_list:
    count += 1
    print(count, size)
    if isinstance(function, str):
        function = [function]
    else :
        function = list(function)
        dropouts = [0.1, 0.1, 0.2]
    model = keras_bitcoin.get_model(texts_train, labels_train, vocab_length,\
                                    max_sentence_size, epochs=epochs,\
                                    batch_size=batch_size, activations_functions=function,\
                                    dropouts=dropouts, verbose = 0)
    loss, accuracy = model.evaluate(texts_test, labels_test, verbose=0)
    preds = model.predict(texts_test)
    if accuracy >= 0.5:
        temp.append(function)
    perfs["_".join(function)] = [accuracy, mean_squared_error(labels_test, preds)]

1 1


C:\Users\33777\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [288]:
perfs

{'softsign_relu_sigmoid_tanh': [0.0, 15.333236874968053]}

In [19]:
df_test = dataset.get_labeled_dataset(number_of_file = 100, from_date = "2017-02-10", date_included = False)

Number of files loaded :  10


In [20]:
preds = keras_bitcoin.get_predictions(list(df_test["text"]), model, vocab_length, max_sentence_size)
df_test["preds"] = list(map(lambda x : int(x), preds))
df_test["correct"] = np.equal(preds, df_test["label"])

In [21]:
dataset.get_prediction_stats(df_test)

Number Correct/Wrong Guess : 5/363
              Accuracy : 1.358695652173913
Invalid sentences count 0
